In [2]:
import os

if not os.path.exists("VoxCeleb_gender"):
    !pip install gdown pydub librosa
    !gdown https://drive.google.com/uc?id=1V3lNnSDv71AJYIj71V4_xD0ir0DrXxEV
    !unzip -q VoxCeleb_gender.zip
    !rm VoxCeleb_gender.zip

else:
    print("Dataset already downloaded.")

Dataset already downloaded.


In [3]:
import librosa
import scipy.signal as ss
import numpy as np
import pandas as pd
import os

def extract_mfcc(file_path, n_mfcc=12, win_length=320, hop_length=160):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    if sr != 16000:
        wave = librosa.resample(wave, sr, 16000)
    mfcc = librosa.feature.mfcc(y=wave, sr=16000, n_mfcc=n_mfcc, window=ss.windows.hamming, win_length=win_length, hop_length=hop_length)
    return mfcc

# Build data in memory

In [4]:
mfcc_coeff_amount = 12

In [5]:

aux = []

for (dirpath, dirnames, filenames) in os.walk("VoxCeleb_gender"):
    for filename in filenames:
        if filename.endswith(".wav"):
            filepath = dirpath + "/" + filename
            aux.append({"file": filepath, "gender": dirpath.split("/")[-1]})

df = pd.DataFrame(aux)

num_males = len(df.loc[df.gender == "males"] )           
num_females = len(df.loc[df.gender == "females"] )           

In [6]:
print("Number of males: ", num_males)
print("Number of females: ", num_females)

Number of males:  3682
Number of females:  2311


In [36]:
df["mfcc"] = pd.Series(name="mfcc", dtype=object)
display(df)

,file,gender,mfcc
0,VoxCeleb_gender/males/255.wav,males,NaN
1,VoxCeleb_gender/males/1908.wav,males,NaN
2,VoxCeleb_gender/males/2965.wav,males,NaN
3,VoxCeleb_gender/males/2733.wav,males,NaN
4,VoxCeleb_gender/males/3091.wav,males,NaN
...,...,...,...
5988,VoxCeleb_gender/females/489.wav,females,NaN
5989,VoxCeleb_gender/females/941.wav,females,NaN
5990,VoxCeleb_gender/females/63.wav,females,NaN
5991,VoxCeleb_gender/females/1820.wav,females,NaN


In [45]:
arr = []

for i in range(len(df)):
    mfcc_coeffs = extract_mfcc(df.loc[i, "file"], n_mfcc=mfcc_coeff_amount)
    for i, coeffs in enumerate(mfcc_coeffs):
        if np.linalg.norm(coeffs) < 1:
            mfcc_coeffs.pop(i)
    arr.append(mfcc_coeffs)

[array([[-266.89233   , -288.64322   , -288.0051    , ..., -685.8922    ,
        -685.8922    , -685.8922    ],
       [  79.592476  ,   26.673538  ,    7.6092668 , ...,    0.        ,
           0.        ,    0.        ],
       [  40.56708   ,   77.108734  ,   91.04158   , ...,    0.        ,
           0.        ,    0.        ],
       ...,
       [  16.491762  ,   12.125517  ,   13.885246  , ...,    0.        ,
           0.        ,    0.        ],
       [   2.269635  ,    6.694189  ,    0.73315436, ...,    0.        ,
           0.        ,    0.        ],
       [   5.2696896 ,   13.936777  ,   11.99409   , ...,    0.        ,
           0.        ,    0.        ]], dtype=float32)]
[array([[-3.6222406e+02, -3.3846970e+02, -3.3541837e+02, ...,
        -8.4559741e+02, -8.4559741e+02, -8.4559741e+02],
       [ 6.5203636e+01,  7.4427147e+01,  7.6193268e+01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.9063938e+01, -5.2346588e+01, -5.3448288e+01, ...,


KeyboardInterrupt: 

In [49]:
print(df.mfcc)


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
5988    NaN
5989    NaN
5990    NaN
5991    NaN
5992    NaN
Name: mfcc, Length: 5993, dtype: object


# EDA

In [14]:
# Get the mean mfcc coefficients for each class
mean_mfcc_males = 0
mean_mfcc_females = 0


for person in df.iterrows():
    for i in range(len(person[1]["mfcc"])//mfcc_coeff_amount):
        if person[1]["gender"] == "males":
            mean_mfcc_males = np.mean(person[1]["mfcc"][i*mfcc_coeff_amount:(i+1)*mfcc_coeff_amount])/num_males
        else:
            mean_mfcc_females = np.mean(person[1]["mfcc"][i*mfcc_coeff_amount:(i+1)*mfcc_coeff_amount])/num_females


TypeError: object of type 'float' has no len()